In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt

In [3]:
# 데이터 로드
data = pd.read_csv("C:\\Users\\LG\\Downloads\\BP_data.csv")

In [6]:
data.isnull().sum()
data = data.fillna(0)

In [8]:
# 불필요 컬럼 제거, 레이블을 예측 타겟으로 설정
X = data.drop(columns=['Patient_Number', 'Blood_Pressure_Abnormality']).values  
y = data['Blood_Pressure_Abnormality'].values.astype(np.float32)         

In [10]:
# 정규화
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [12]:
# 시퀀스 형태로 변환
def split_sequences(sequences, n_steps):
    X, y = [], []
    for i in range(len(sequences)):
        end_ix = i + n_steps
        if end_ix > len(sequences):
            break
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [14]:
# Blood_Pressure_Abnormality를 붙여서 시퀀스 분할
full_data = np.hstack((X, y.reshape(-1, 1)))
X_seq, y_seq = split_sequences(full_data, n_steps=5)

In [16]:
# Train/Test 분할
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)


In [18]:
# Tensor 변환
X_train = torch.tensor(X_train, dtype=torch.float32).permute(0, 2, 1)  # (batch, channels, seq_len)
X_test = torch.tensor(X_test, dtype=torch.float32).permute(0, 2, 1)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

In [20]:
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

In [22]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [26]:
#모델 정의: Conv1D 회귀 모델
class BPCNN(nn.Module):
    def __init__(self):
        super(BPCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=13, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32 * 5, 64)
        self.fc2 = nn.Linear(64, 1)  # 회귀 출력

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        return self.fc2(x)


In [28]:
#학습 및 평가
model = BPCNN()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train_losses = []

# Training loop
for epoch in range(30):
    model.train()
    running_loss = 0.0
    for xb, yb in train_loader:
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    train_losses.append(running_loss / len(train_loader))
    print(f"Epoch {epoch+1}, Loss: {train_losses[-1]:.4f}")

Epoch 1, Loss: 0.2697
Epoch 2, Loss: 0.2190
Epoch 3, Loss: 0.1894
Epoch 4, Loss: 0.1695
Epoch 5, Loss: 0.1574
Epoch 6, Loss: 0.1458
Epoch 7, Loss: 0.1342
Epoch 8, Loss: 0.1251
Epoch 9, Loss: 0.1147
Epoch 10, Loss: 0.1077
Epoch 11, Loss: 0.0981
Epoch 12, Loss: 0.0870
Epoch 13, Loss: 0.0816
Epoch 14, Loss: 0.0719
Epoch 15, Loss: 0.0617
Epoch 16, Loss: 0.0557
Epoch 17, Loss: 0.0491
Epoch 18, Loss: 0.0448
Epoch 19, Loss: 0.0398
Epoch 20, Loss: 0.0359
Epoch 21, Loss: 0.0325
Epoch 22, Loss: 0.0286
Epoch 23, Loss: 0.0239
Epoch 24, Loss: 0.0207
Epoch 25, Loss: 0.0180
Epoch 26, Loss: 0.0157
Epoch 27, Loss: 0.0138
Epoch 28, Loss: 0.0124
Epoch 29, Loss: 0.0116
Epoch 30, Loss: 0.0102


In [30]:
# Evaluation
model.eval()
preds = []
actuals = []
with torch.no_grad():
    for xb, yb in test_loader:
        out = model(xb)
        preds.extend(out.squeeze().cpu().numpy())
        actuals.extend(yb.squeeze().cpu().numpy())

mse = mean_squared_error(actuals, preds)
r2 = r2_score(actuals, preds)
print(f"Test MSE: {mse:.4f}")
print(f"R² Score: {r2:.4f}")

Test MSE: 0.2554
R² Score: -0.0220
